In [1]:
from models.BYOL2_model import BYOL2
from data.custom_transforms import BatchTransform, ListToTensor, PadToSquare, SelectFromTuple
from data.pairs_dataset import PairsDataset, pair_collate_fn
from data.custom_transforms import RandomLiquify, RandomMovement, HueAdjust, SaturationAdjust

from data.multi_pairs_dataset import MultiPairsDataset
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
import torch
from torch.utils.data import Subset
import torchvision.models as models
import torchvision.transforms as T

import warnings


In [2]:
ecommerce = {"images" : '/home/vision/smb-datasets/ecommerce/images/train/',
             "sketches" : '/home/vision/smb-datasets/ecommerce/ecommerce_pidinet/'}

ecommerce_pidinet_gaussian3 = {"images" : '/home/vision/smb-datasets/ecommerce/images/train/',
                                "sketches" : '/users/btorres/databases/ecommerce/pidinet_gaussian1/'}

ecommerce_pidinet_color = {"images" : '/users/btorres/databases/ecommerce/images/train/',
                           "sketches" : '/users/btorres/databases/n-ecommerce/pidinet_color_quanta16_layer/'}

ecommerce_liquify = {"images" : '/home/vision/smb-datasets/ecommerce/images/train/',
             "sketches" : '/users/btorres/databases/ecommerce/pidinet_liquify3/'}

sketchy = {"images" : '/home/btorres/databases/sketchy/photo/tx_000100000000/',
           "sketches" : '/home/btorres/databases/sketchy/sketch/tx_000100000000/'}

color = {"images" : '//users/btorres/databases/n-ecommerce/train/',
         "sketches" : '/users/btorres/databases/n-ecommerce/pidinet_color_quanta16_layer_10hues/'}

ecommerce_segment = {"images" : "/users/btorres/database/ecommerce/segment/train/",
           "sketches" : "/users/btorres/database/ecommerce/segment/sketches/"}


#save_file = 'bimodal_byol_shoes_utils/checkpoints/ecommerce/self_bimodal_byol_sketchy_ecommerce_pidinet_batch70_liquify5nucleusInOut_checkpoint_{}epochs.pt'
save_file = 'bimodal_byol_shoes_utils/checkpoints/Color/pretrained_ecommerce_bestSketch_filtros_segment_{}epoch.pt'
pretrained_model = 'bimodal_byol_shoes_utils/checkpoints/sketchy/self_bimodal_byol_ecommerce_liquifyOnline90_randomMovement50_batch105_1epochs.pt'
#pretrained_model = 'bimodal_byol_shoes_utils/checkpoints/sketchy/self_bimodal_byol_pretrained_sketchy_batch80_resnet4blocks_4epochs.pt'





device = "cuda:0"
batch_size = 110
torch.cuda.empty_cache()

In [3]:
#ecommerce_dataset = PairsDataset(
#    ecommerce["images"],
#    ecommerce["sketches"]
#)

#ecommerce_dataset_pidinet = PairsDataset(
#    ecommerce_pidinet_gaussian3["images"],
#    ecommerce_pidinet_gaussian3["sketches"]
#)

#ecommerce_pidinet_color16 = PairsDataset(
#    ecommerce_pidinet_color["images"],
#    ecommerce_pidinet_color["sketches"]
#)

#ecommerce_color16_10hues = PairsDataset(
#    color["images"],
#    color["sketches"]
#)

#ecommerce_dataset_liquify = PairsDataset(
#    ecommerce_liquify["images"],
#    ecommerce_liquify["sketches"]
#)

#sketchy_dataset = MultiPairsDataset(
#    sketchy["images"],
#    sketchy["sketches"],
#    separator="-"
#)

ecommerce_dataset_segment = PairsDataset(
    ecommerce_segment["images"],
    ecommerce_segment["sketches"]
)


#ratio80 = RatioDataset(batch_size, 0.8, sketchy_dataset, ecommerce_dataset)
#ratio20 = RatioDataset(batch_size, 0.2, sketchy_dataset, ecommerce_dataset)


In [4]:

# resize the images for the net 
transforms_1 = T.Compose([
    BatchTransform(SelectFromTuple(0)),
    BatchTransform(PadToSquare(255)),
    BatchTransform(T.Resize((224,224))),
    ListToTensor(device, torch.float),
])
transforms_2 = T.Compose([
    BatchTransform(SelectFromTuple(1)), #sketch
    BatchTransform(PadToSquare(255)),
    BatchTransform(T.Resize((224,224))),
    #BatchTransform(SaturationAdjust(prob=0.5, saturation_shift=0.06)),
    BatchTransform(RandomMovement(prob=0.5)),
    BatchTransform(RandomLiquify(prob=0.5, size=(224, 224), curve=1.1, strength=0.03, steps=10)),
    ListToTensor(device, torch.float),
])

#BatchTransform(RandomLiquify(prob=0.9, size=(224,224))),
#BatchTransform(RandomMovement(0.5)),

train_loader = torch.utils.data.DataLoader(
    ecommerce_dataset_segment,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=pair_collate_fn,
    num_workers=4
)

"""
# function for loading the images as batchs
Loaders = []
for i in range(len(Datasets)):
    train_loader = torch.utils.data.DataLoader(
        Datasets[i],
        batch_size=batch_size,
        shuffle=True,
        collate_fn=pair_collate_fn,
        num_workers=4
    )
    Loaders.append(train_loader)
"""

'\n# function for loading the images as batchs\nLoaders = []\nfor i in range(len(Datasets)):\n    train_loader = torch.utils.data.DataLoader(\n        Datasets[i],\n        batch_size=batch_size,\n        shuffle=True,\n        collate_fn=pair_collate_fn,\n        num_workers=4\n    )\n    Loaders.append(train_loader)\n'

In [5]:
#encoder = timm.create_model('nf_resnet50', pretrained=True)
encoder = models.resnet50(weights="IMAGENET1K_V2")
#encoder = models.resnet.ResNet( models.resnet.Bottleneck, [3,4,6,0])
#encoder = models.resnet._resnet(models.resnet.Bottleneck, [3,4,6,0], None, False)

#encoder.layer4 = encoder.layer4[0:1]
#print(encoder)

print(encoder)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
#encoder = models.resnet50()
#encoder.load_state_dict(torch.load('../checkpoints/resnet50_byol_quickdraw_128_1000_v3.pt'))

empty_transform = T.Compose([])
epochs = 2
epoch_size = len(train_loader)

learner = BYOL2(
    encoder,
    image_size=224,
    hidden_layer="avgpool",
    augment_fn=empty_transform,
    cosine_ema_steps=epochs*epoch_size,
)

# se agregan las transformaciones a la red
learner.augment1 = transforms_1
learner.augment2 = transforms_2

# optimizador
opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

In [7]:

learner.load_state_dict(torch.load(pretrained_model))
torch.save(learner.state_dict(), save_file.format(epochs))
#learner.load_state_dict(torch.load( save_file.format(epochs)))

learner = learner.to(device)

#if torch.cuda.device_count() > 1:
#    print("Let's use", torch.cuda.device_count(), "GPUs!")
#    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#    learner = torch.nn.DataParallel(learner, device_ids=[0, 1])
#learner = learner.to(device)


In [8]:

learner.train()
filehandler = open('weightFiles/filehandler.txt', 'w')
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    running_loss = np.array([], dtype=np.float32)
    for epoch in range(epochs):
        i = 0

        #for images in Loaders[int(epoch/epochsForLoader)]:
        for images in train_loader:
            loss = learner(images)#.to('cuda', dtype=torch.float)
            opt.zero_grad()
            loss.backward()
            opt.step()
            learner.update_moving_average()
            running_loss = np.append(running_loss, [loss.item()])
            sys.stdout.write('\rEpoch {}, batch {} - loss {:.4f}'.format(epoch+1, i+1, np.mean(running_loss)))
            filehandler.write('Epoch {}, batch {} - loss {:.4f}\n'.format(epoch+1, i+1, np.mean(running_loss)))
            filehandler.flush()
            i += 1
            #if i%(epoch_size/2)==0:
            #    torch.save(learner.state_dict(), save_file.format(epochs))

        #if epoch % 2 == 0:
        torch.save(learner.state_dict(), save_file.format(epoch + 1))
        running_loss = np.array([], dtype=np.float32)
        sys.stdout.write('\n')
filehandler.close()

Epoch 1, batch 47 - loss 0.1700
Epoch 2, batch 47 - loss 0.1529
